In [ ]:
import os, re, matplotlib, pandas, collections, importlib, sys, pickle, HTSeq, random, bisect, dill
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import sklearn
from sklearn.neighbors.kde import KernelDensity

#os.chdir('/Users/dp/pma/dataAndScripts/clip/miseq/v2all/')
sys.path.append('/Users/dp/pma/')
sys.path.append('/Users/dp/pma/dataAndScripts/clip/')

import sameRiver
import sameRiver.peakFinder
import sameRiver.peakSeqFinder
import sameRiver.statsForCounts
import sameRiver.rnaDataFileMaker
import sameRiver.exp

importlib.reload(sameRiver.peakFinder)
importlib.reload(sameRiver.peakSeqFinder)
importlib.reload(sameRiver.statsForCounts)
importlib.reload(sameRiver.countsO)
importlib.reload(sameRiver.rnaDataFileMaker)
top = top_dir = '/Users/dp/pma/dataAndScripts/clip/meta/'
os.chdir(top)

In [ ]:
# Section 1

# Load signal and RNA data.
#with open('/Users/dp/pma/dataAndScripts/clip/meta/data/signal_rnas_m2.data', 'rb') as f:
#    ssr = dill.load(f)
importlib.reload(sameRiver.rnaDataFileMaker)
maker = sameRiver.rnaDataFileMaker.rnaDataFileMaker()
RNAs = maker.make_from_gtf_file(
    gtf_filename='/Users/dp/pma/repeats_and_genome.gtf',  # New mapping method gtf.
    #gtf_filename='/opt/genomes/repeats_and_ensembl_release94_GRCh38/combined_tsl1andNA.gtf',  # Old mapping gtf.
    output_data_filename=f"{top}/data/rna.data",
)
exp = sameRiver.exp.exp(name='meta', file_paths=sameRiver.exp.exp.autogenerate_paths(top))
#RNAs.example(10)


In [ ]:
# Section 2

# Determine the preferential binding locations for all proteins across RNAs,
# if they exist. peakFinder.find_peaks(ssr) can use a pvals*.xlsx file
# to limit to RNAs being bound significantly by some model. But
# we prefer to just output everything, and can filter later.
# This falls in the sequence ssr -> peak_locations.xlsx, and its output
# is used by peakSignalFinder.

import sameRiver.peakFinder
import sameRiver.peakSeqFinder
import sameRiver.peakSignalFinder

importlib.reload(sameRiver.peakFinder)
importlib.reload(sameRiver.peakSeqFinder)
importlib.reload(sameRiver.peakSignalFinder)

top_dir = top

pk = sameRiver.peakFinder.peakFinder(
    scheme_filename=f'{top}/scheme.xlsx',
    rnas_obj=RNAs)

print("Finding peaks...")
pk.find_peaks(
    method='convolve',
    excel_of_target_RNAs=f'{top}/tables/pvals_per_read.xlsx',
    bedgraph_folder=f'{top}/beds/read_start/',
    #top_n_targets=500,
    p_cutoff=10, #only_do=['FBL'],
    outfile=f'{top}/data/peak_locations.xlsx',
    )

print(f"Found peaks for {len(pk.peaks.keys())} protein(s).")


In [ ]:
#sys.path.append('/Users/dp/pma/')

import collections
import sameRiver.peakSignalFinder
importlib.reload(sameRiver.peakSignalFinder)

# Section 3

# Write an xlsx file in {top}/data for each protein in peak_locations.xlsx.
# For each protein X in peak_locations.xlsx:
# write a file containing signal for each bedgraph replicate file,
# across all proteins, at the peak locations of protein X.
# This requires ssr -> peakFinder.find_peaks(ssr) -> peak_locations.xlsx,
# and the output files are input to statsForCountsNB to determine p values,
# and input to peakSeqFinder for fasta files (data/seq/) of significant peak regions.

#os.chdir(top_dir)
sg = sameRiver.peakSignalFinder.peakSignalFinder(scheme_filename=f'{top}/scheme.xlsx', rnas_obj=RNAs)

sg.find_signal_at_peaks(
    excel_of_peak_locations=f'{top}/data/peak_locations.xlsx',
    bedgraph_folder=f'{top}/beds/read_start/', #only_do=['FBL', 'SF3B1', 'PCBP1'],
    output_folder=f'{top}/data/')

In [ ]:
# Section 4

# Evaluate p values for signal at peak locations.
# This follows peakSignalFinder()
# ./data/signal_at_{gene}_peak_locations.xlsx -> 
# ./data/pvals_for_signal_at_{}_peaks_per_read.xlsx
# The output is used by peakSeqFinder.

import importlib
import sameRiver
import sameRiver.metadata
import sameRiver.negativeCounts
import sameRiver.positiveCounts
import sameRiver.scheme
import sameRiver.statsForCountsNB
importlib.reload(sameRiver.positiveCounts)
importlib.reload(sameRiver.negativeCounts)
importlib.reload(sameRiver.statsForCounts)
importlib.reload(sameRiver.statsForCountsNB)
importlib.reload(sameRiver.scheme)

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

def write_signal_at_peak_locations(gene, top_dir):
    
    counts = f'{top}/data/signal_at_{gene}_peak_locations.xlsx'
    
    if not os.path.exists(counts):
        return
    
    positive_metadata = Namespace(
        top_dir = top,
        scheme_file = f'{top}/scheme.xlsx',
        ann_counts_file = f'{top}/data/signal_at_{gene}_peak_locations.xlsx',
        bed_file_dir = f'{top}/beds/',
        positive_proteins = [
            'CAPNS2', #'CCIN', 
            'CDK4', 'CHMP3',
            'DCTN6', #'EPB41L5', 
            'ETS2', 'IDE',
            'ITPA', 'TPGS2', 'UBA2',
            'FBL', #'HCT116', 
            'hnRNPC',
            'SF3B1',
            'PCBP1', 'PCBP1:100P', 'PCBP1:100Q',
            'PCBP1:dKH', 'CELF1',
            'Rbfox1', 'Rbfox2', 'hnRNPD',
        ]
    )

    negative_metadata = Namespace(
        top_dir = top,
        scheme_file_with_random_proteins = f'{top}/scheme.xlsx',
        ann_counts_file = f'{top}/data/signal_at_{gene}_peak_locations.xlsx',
        bed_file_dir = f'{top}/beds/',
        random_proteins = [
            'CAPNS2', #'CCIN',
            'CDK4', 'CHMP3',
            'DCTN6', #'EPB41L5',
            'ETS2', 'IDE',
            'ITPA', 'TPGS2', 'UBA2',
        ])

    def make_negatives():
        negatives = sameRiver.negativeCounts.negativeCounts(
            metadata=negative_metadata, xl_rate_fname=top_dir + '/percentCrosslinked.xlsx')
        # Optional: write_txt=True to write some txt's of the data.
        negatives.save(fname=f'{top}/data/signal_at_{gene}_peaks_negatives.dill', write_object=True, write_txt=False)

    def make_positives():
        positives = sameRiver.positiveCounts.positiveCounts(
            metadata=positive_metadata, xl_rate_fname=top_dir + '/percentCrosslinked.xlsx')
        positives.save(fname=f'{top}/data/signal_at_{gene}_peaks_positives.dill', write_object=True, write_txt=False)
        
    # If never run before:
    make_negatives()

    # If never run before:
    make_positives()

    return True

def write_pvals(gene):
    positives = sameRiver.positiveCounts.positiveCounts.load(fname=f'{top}/data/signal_at_{gene}_peaks_positives.dill')
    negatives = sameRiver.negativeCounts.negativeCounts.load(fname=f'{top}/data/signal_at_{gene}_peaks_negatives.dill')
   # print(positives.raw_reads_per_gene.df)
    print('top=', top)
    #nb = sameRiver.statsForCountsNB.statsForCountsNB.load()
    nb = sameRiver.statsForCountsNB.statsForCountsNB(
        negatives=negatives, positives=positives,
        data_dir=f'{top}/data/')
    if nb.calculate_pvalues(which='per_read', apply_bh_adjust=True):
        nb.write_pvals_single_file(which='per_read', outfname=f'{top}/data/pvals_for_signal_at_{gene}_peaks_per_read.xlsx')
    if nb.calculate_pvalues(which='per_protein', apply_bh_adjust=True):
        nb.write_pvals_single_file(which='per_protein', outfname=f'{top}/data/pvals_for_signal_at_{gene}_peaks_per_protein.xlsx')


def combine_pvals_into_single_file(top):

    _d = {}
    for prot in set(scheme.proteins):
        print(prot)
        fname = f'{top}/data/pvals_for_signal_at_{prot}_peaks_per_read.xlsx'
        if not os.path.exists(fname):
            continue
        df = pandas.read_excel(fname, index_col=0)
        if prot in df:
            _d[prot] = dict(zip(df.index, df[prot]))
        else:
            print(f"Did not find {prot} in {fname}")
    
    df = pandas.DataFrame(_d)
    df['gene_name'] = df.index
    df.fillna(1., inplace=True)
    
    print('====')
    print(df.head())
    df.to_excel(f'{top}/tables/pvals_at_peaks_per_read.xlsx')

    _d = {}
    for prot in [_ for _ in scheme.proteins if _ in df]:
        fname = f'{top}/data/pvals_for_signal_at_{prot}_peaks_per_protein.xlsx'
        
        if not os.path.exists(fname):
            continue
        
        df = pandas.read_excel(fname, index_col=0)
        _d[prot] = dict(zip(df.index, df[prot]))
        
    df = pandas.DataFrame(_d)
    df.fillna(1., inplace=True)
    df['gene_name'] = df.index

    print(df.head(1))
    df.to_excel(f'{top}/tables/pvals_at_peaks_per_protein.xlsx')

    
scheme = sameRiver.scheme.scheme(f'{top}/scheme.xlsx')

print('----')

for prot in list(scheme.proteins):
    if write_signal_at_peak_locations(prot, top_dir):
        write_pvals(prot)
    else:
        print(f'failed for {prot}')
        
combine_pvals_into_single_file(top)



In [ ]:
homer_command = """
# For some reason, homer1 works while homer2 doesn't.

export PATH=$PATH:/Users/dfporter/homer/bin/

./bin/findMotifs.pl ../pma/tables/seqs/hnRNPC.fa  fasta ./hnRNPC -fasta ../pma/tables/seqs/randoms_for_hnRNPC.fa -rna -homer1 
./bin/findMotifs.pl ../pma/tables/seqs/FBL.fa  fasta ./FBL -fasta ../pma/tables/seqs/randoms_for_FBL.fa  -rna -homer1
"""


In [ ]:
"""
Assemble outputs (made in write_fasta_for_homer.ipynb) into one file.
"""

import glob
from pathlib import Path

os.system('export PATH=$PATH:/Users/dp/homer/bin/')
print('export PATH=$PATH:/Users/dp/homer/bin/')
for fname in glob.glob(top_dir+'/data/seqs/*fa'):
    if 'random' in fname:
        continue

    gene = os.path.basename(fname).split('.fa')[0]
    random_fname = os.path.dirname(fname) + '/randoms_for_{0}.fa'.format(gene)
    homer = '/Users/dfporter/homer/bin/findMotifs_edit.pl'
    cmd = "{} {} fastq ./{} -fasta {} -rna -homer1".format(homer, fname, gene, random_fname)
    #print(cmd)
    #os.system(cmd)
    
class logoAssembly:
    
    def __init__(self, homer_dir='./'):
        self.homer_dir = homer_dir
    
    def find_all_svg_motif1_logos(self):
        print(f'{self.homer_dir}/*/motif1.logo.svg')
        svgs_motif1, svgs_motif2 = {}, {}
        for path in glob.glob(f'{self.homer_dir}/*/*/motif1.logo.svg'):
            with open(path) as f:
                svgs_motif1[os.path.dirname(path)] = f.readlines()
                
        for path in glob.glob(f'{self.homer_dir}/*/*/motif2.logo.svg'):
            print(path)
            with open(path) as f:
                svgs_motif2[os.path.dirname(path)] = f.readlines()
        #print(svgs)
        
        header = """<HTML><HEAD><TITLE>Homer de novo Motif Results</TITLE></HEAD>
<BODY>
<H1>Homer <i>de novo</i> Motif Results </H1>
"""
        print(os.getcwd())
        comb = open(self.homer_dir+'/all_logos.html', 'w')
        comb.write(header)
        for path, lines in svgs_motif1.items():
            
            comb.write('<H2>{}</H2>\n'.format(os.path.dirname(path)))
            comb.write(''.join(lines))
            if path in svgs_motif2:
                comb.write(''.join(svgs_motif2[path]))                
        comb.close()
        
logos = logoAssembly(
    homer_dir='/Users/dp/homer/top10Ktargets_5_min_height_no_Pvalue_cutoffs_20k_controlSeqs_mRNA_only_old_mapping_eg_repeats_first_Thursday'
)
logos.find_all_svg_motif1_logos()